In [41]:
import json
import re
import os
import pandas as pd
import plotly.express as px
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from pprint import pprint

In [42]:
load_dotenv("../.env")

llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0.3,
    openai_api_key = os.getenv("OPENAI_API_KEY_AIFFEL")
)

In [43]:
prompt = PromptTemplate(
    input_variables=["user_query", "bot_response"],
    template="""
다음은 사용자와 챗봇 간의 책 추천 대화입니다.

[사용자 질문]
{user_query}

[챗봇 응답]
{bot_response}

이 대화를 아래의 5개 기준으로 평가해주세요. 각 항목에 대해 1~5점 척도로 점수를 매기고, 짧은 평가 이유도 함께 작성해주세요. 결과는 JSON 형식으로 작성해주세요.

[평가 기준 및 점수 가이드라인]

1. 사용자 의도 반영성
- 1점: 사용자 질문과 전혀 무관한 응답
- 2점: 사용자 의도를 잘못 해석함
- 3점: 의도를 일부 반영했으나 부족함
- 4점: 대부분 반영했으나 세부적으로 아쉬움 있음
- 5점: 사용자 의도를 정확하고 명확하게 반영함

2. 추천 도서의 적합성
- 1점: 질문과 관련 없는 도서 추천
- 2점: 관련성 낮은 도서 추천
- 3점: 관련성은 있으나 일반적인 수준
- 4점: 꽤 적절한 도서 추천
- 5점: 질문에 매우 적합한 도서 추천

3. 대화 흐름의 자연스러움
- 1점: 맥락이 끊기거나 매우 어색한 응답
- 2점: 다소 어색하거나 문장이 부자연스러움
- 3점: 전반적으로는 자연스러우나 약간 어색한 부분 있음
- 4점: 대부분 자연스럽고 매끄러움
- 5점: 매우 자연스럽고 대화 흐름이 매끄러움

4. 추천 이유 설명력
- 1점: 이유 설명이 없거나 매우 부족함
- 2점: 간단한 설명만 제공됨
- 3점: 이해는 가능하나 설득력이 약함
- 4점: 적절한 설명이 포함되어 있음
- 5점: 논리적이고 설득력 있는 이유 설명이 포함됨

5. 정보의 정확성
- 1점: 명백한 오류나 잘못된 정보 포함
- 2점: 일부 사실과 다른 정보 포함
- 3점: 다소 애매하거나 불확실한 정보 포함
- 4점: 전반적으로 정확하나 일부 불명확한 표현 존재
- 5점: 정보가 명확하고 정확함

[결과 예시]

[
  {{
    "항목": "사용자 의도 반영성",
    "점수": ,
    "이유": ""
  }},
  {{
    "항목": "추천 도서의 적합성",
    "점수": ,
    "이유": ""
  }},
  {{
    "항목": "대화 흐름의 자연스러움",
    "점수": ,
    "이유": ""
  }},
  {{
    "항목": "추천 이유 설명력",
    "점수": ,
    "이유": ""
  }},
  {{
    "항목": "정보의 정확성",
    "점수": ,
    "이유": ""
  }}
]
"""
)

In [44]:
# LLM Chain 구성
evaluation_chain = prompt | llm | StrOutputParser()

# 모델 단일 응답 평가

In [ ]:
# 테스트 평가 수행
user_query = "스트레스를 줄이고 싶은데, 편안하게 읽을 수 있는 책 추천해줘."
bot_response = """
다음 책을 추천드려요:  
📘 『죽고 싶지만 떡볶이는 먹고 싶어』 - 자존감이 낮고 힘든 하루를 살아가는 분들에게 감정적인 위로를 주는 책입니다.  
📘 『아주 작은 습관의 힘』 - 스트레스 관리와 생활 습관을 개선하는 데 도움을 주는 실용적인 책입니다.
"""

result = evaluation_chain.invoke({
    "user_query": user_query,
    "bot_response": bot_response
})

print(result)

result

In [ ]:
samples = []

for query, response in zip(user_queries, bot_responses):
    samples.append({
        "user_query": query,
        "bot_response": response
    })

samples

In [ ]:
multi_eval_chain = prompt | llm | StrOutputParser()
results = multi_eval_chain.batch(samples)

for i, res in enumerate(results):
    print(f"\n🔹 샘플 {i+1} 평가 결과:\n{res}")

# A/B Test (Comparision with Crema AI from yes24)

In [32]:
def ab_test(df1, df2, llm):
    # A/B Test 데이터셋 작성
    questions = df1["question"]
    responses_a = df1["answer"]
    responses_b = df2["answer"]

    ab_dataset = []
    for question, response_a, response_b in zip(questions, responses_a, responses_b):
        ab_dataset.append({
            "question": question,
            "response_a": response_a,
            "response_b": response_b
        })
    
    # A/B Test Prompt
    ab_prompt = PromptTemplate(
    input_variables=["question", "response_a", "response_b"],
    template="""다음은 한 사용자의 질문과 두 개의 챗봇 응답입니다.
    
    [사용자 질문]
    {question}
    
    [응답 A]
    {response_a}
    
    [응답 B]
    {response_b}
    
    위 두 응답을 비교하여, 사용자 입장에서 더 적절하고 유용한 응답을 선택해주세요. 이유도 함께 설명해주세요.
    이유에는 [응답 A] 와 [응답 B] 의 간략한 설명이 포함되어야 하고, 두가지를 비교한 이유가 포함되어야 합니다.
    
    결과는 아래 형식으로 JSON으로 작성해주세요:
    {{
    "우수 응답": "A" 또는 "B",
    "이유": "선택한 이유를 명확히 기술"
    }}"""
    )
    
    # A/B Test Chain 구성
    ab_chain = ab_prompt | llm | StrOutputParser()

    # 결과 추출
    results = ab_chain.batch(ab_dataset)
    parsed = [clean_and_parse_json_block(r) for r in results]
    results_df = pd.DataFrame(parsed)
    results_df["questions"] = questions
    results_df["response_a"] = responses_a
    results_df["response_b"] = responses_b

    return results_df

    
def clean_and_parse_json_block(block: str):
    try:
        # 1. ```json 시작과 마지막 ``` 제거
        block = block.strip()
        block = re.sub(r"^```json\s*", "", block)
        block = re.sub(r"\s*```$", "", block)

        # 2. 줄바꿈(\n) 제거 → JSON 문자열로 만들기
        json_str = block.replace("\n", "")
        
        # 3. JSON 파싱
        return json.loads(json_str)
    
    except Exception as e:
        return {"우수 응답": None, "이유": f"[Error parsing]: {e}"}
    


In [12]:
# A/B Test 시각화 함수
def draw_ab_result(results_df, model_name_list = ["A", "B"]):
    ab_counts = results_df["우수 응답"].value_counts().reset_index()
    ab_counts.columns = ["응답", "선택 수"]

    label_map = {
        "A": model_name_list[0],
        "B": model_name_list[1]
    }
    ab_counts["응답 라벨"] = ab_counts["응답"].map(label_map)

    fig = px.bar(
        ab_counts,
        x = "응답 라벨",
        y = "선택 수",
        color = "응답",
        text = "선택 수",
        title = f"A/B Test Result : {model_name_list[0]} & {model_name_list[1]}",
        category_orders = {"응답": ["A", "B"]}
    )

    fig.show()

In [ ]:
crema = pd.read_csv(f_path_crema)
model1 = pd.read_csv()
model2 = pd.read_csv()
model3 = pd.read_csv()
model5 = pd.read_csv()

In [33]:
results_1 = ab_test(model5, model1, llm, )
draw_ab_result(results_1, ["model5", "model1"])

In [28]:
results_3 = ab_test(model5, model3, llm, )
draw_ab_result(results_3, ["model5", "model3"])

In [29]:
results_4 = ab_test(model5, crema, llm, )
draw_ab_result(results_4, ["model5", "cremaAI"])